In [48]:
import pandas as pd
import numpy as np
import sqlite3
import json
import math

,x,y,pull_x,pull_y,is_foot
hold_index,,,,,
390,0.666667,0.333333,0.0,-0.5,1
392,0.666667,1.666667,0.0,-0.5,1
394,0.666667,3.000000,0.0,-0.5,0
396,0.666667,4.333333,0.0,-0.5,0
399,0.666667,6.333333,0.0,-0.5,0


In [4]:
fdf.head()

,wall_id,angle,holds,grade,quality,ascents
id,,,,,,
climb-5f68ab77af17,wall-443c15cd12e0,40,"[[570, 0], [512, 1], [523, 2], [576, 2], [578,...",15.9936,2.97925,5011
climb-89a4f1804603,wall-443c15cd12e0,40,"[[534, 0], [607, 0], [687, 1], [504, 2], [611,...",18.0034,2.98120,4095
climb-a7649d8bb38b,wall-443c15cd12e0,40,"[[699, 0], [684, 1], [691, 1], [747, 2], [787,...",15.9897,2.96407,3896
climb-b0df21d76511,wall-443c15cd12e0,40,"[[641, 0], [747, 0], [792, 1], [680, 2], [752,...",13.0074,2.97686,3673
climb-ae7536aebc95,wall-443c15cd12e0,40,"[[734, 0], [582, 1], [615, 2], [684, 2], [619,...",15.0011,2.98169,3660


In [55]:
def apply_wall_angle(angle, y):
    """Function for turning a single set of hold features into 3d features"""
    rads = math.radians(angle)
    y2 = y * math.cos(rads)
    z = y * math.sin(rads)

    return (y2,z)

def _3d_point_cloud(angle, hl, to_length=30):
    """Converts a list of hold indices + roles to a 2d-point cloud and pads the sequence by adding random NULL holds."""
    pad_length = to_length-len(hl)
    padding = list(zip(list(np.random.choice(hdf.index,size=pad_length)), [4]*pad_length))
    padded_list = hl + padding

    _3d_features = []
    for hold in padded_list:
        role = hold[1]
        features = hdf.loc[hold[0]]
        [x, y, pull_x, pull_y, is_foot] = [f for f in features]
        y, z = apply_wall_angle(angle, y)
        pull_y, pull_z = apply_wall_angle(angle, pull_y)
        _3d_features.append([x, y, z, pull_x, pull_y, pull_z, is_foot, role])
    return np.array(_3d_features, dtype=np.float32) 

features_3d = []
for _, row in fdf.iterrows():
    try:
        features_3d.append(_3d_point_cloud(row['angle'],row['holds']))
    except:
        continue
features_3d = np.array(features_3d)
features_3d.shape

(29705, 30, 8)

In [53]:
features_3d[0]

array([[ 4.        ,  2.8088295 ,  1.8054808 ,  0.        , -0.38302222,
        -0.24620193,  0.        ,  0.        ],
       [ 2.6666667 ,  8.937185  ,  5.744712  ,  0.        , -0.38302222,
        -0.24620193,  0.        ,  1.        ],
       [ 3.        ,  3.5748742 ,  2.2978847 ,  0.        , -0.38302222,
        -0.24620193,  0.        ,  2.        ],
       [ 4.        ,  5.8730073 ,  3.7750964 ,  0.        , -0.38302222,
        -0.24620193,  0.        ,  2.        ],
       [ 4.        ,  6.8944    ,  4.431635  ,  0.        , -0.38302222,
        -0.24620193,  0.        ,  2.        ],
       [ 4.3333335 ,  6.639052  ,  4.2675004 ,  0.        , -0.38302222,
        -0.24620193,  0.        ,  2.        ],
       [ 5.        ,  5.106963  ,  3.2826924 ,  0.        , -0.38302222,
        -0.24620193,  0.        ,  2.        ],
       [ 5.        ,  8.171141  ,  5.252308  ,  0.        , -0.38302222,
        -0.24620193,  0.        ,  2.        ],
       [ 3.3333333 ,  0.25534815